<a href="https://colab.research.google.com/github/tarunbalajiks/Speech-Emotion-Recognition/blob/main/DeployWebApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pyngrok

In [5]:
!ngrok config add-authtoken TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
import keras
import librosa
import numpy as np
from keras.layers import Input, Conv1D, Flatten, Dense, Dropout, Activation, MaxPooling1D
from keras.models import Model
from flask import Flask, request, redirect, render_template_string, send_from_directory
from werkzeug.utils import secure_filename
import os
from io import BytesIO
import base64
import os
from pyngrok import ngrok

In [7]:
''' Class to Make Predicitions '''

class LivePredictions:
    def __init__(self):
        self.path = "/content/Emotion_Voice_Detection_Model.h5"

        input_layer = Input(shape=(40, 1))
        x = Conv1D(64, 5, padding='same')(input_layer)
        x = Activation('relu')(x)
        x = Dropout(0.2)(x)
        x = Flatten()(x)
        x = Dense(8)(x)
        output_layer = Activation('softmax')(x)
        self.model = Model(inputs=input_layer, outputs=output_layer)
        self.model.load_weights(self.path)

    @staticmethod
    def convert_class_to_emotion(pred):
        label_conversion = {'0': 'Neutral',
                            '1': 'Calm',
                            '2': 'Happy',
                            '3': 'Sad',
                            '4': 'Angry',
                            '5': 'Fearful',
                            '6': 'Disgust',
                            '7': 'Surprised'}

        for key, value in label_conversion.items():
            if int(key) == pred:
                label = value

        return label

    def make_predictions(self, file):
        data, sampling_rate = librosa.load(file)
        mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=40).T, axis=0)
        x = np.expand_dims(mfccs, axis=0)
        predictions = self.model.predict(x, verbose=0)
        predicted_class = np.argmax(predictions)
        predicted_probability = max(predictions[0])*100
        return self.convert_class_to_emotion(predicted_class), round(predicted_probability,2)

In [8]:
''' Check if model is working correctly '''

predection = LivePredictions()
predection.model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 40, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 40, 64)              │             384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 40, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 40, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2560)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 8)                   │          20,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 8)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,872 (81.53 KB)

 Trainable params: 20,872 (81.53 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
''' Recommender System '''

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Define a function to get similar activities
def get_similar_activities(predicted_emotion, top_n=10):
    activity_name = emotion_activity_map[predicted_emotion]
    base_index = data[data["Activity_Name"] == activity_name].index[0]
    similarities = cosine_similarity(tfidf_matrix[base_index], tfidf_matrix).flatten()

    # Sort by similarity and get top N
    data["Similarity"] = similarities
    similar_activities = data.sort_values(by="Similarity", ascending=False).iloc[1:top_n+1]
    wanted_activities = similar_activities[["Activity_Name", "Description"]]
    wanted_activities = wanted_activities.sample(3)
    return wanted_activities.to_dict('records')

''' Basic Emotion-Activity Mapping '''
emotion_activity_map = {
    "Neutral": "Debating",
    "Calm": "Yoga",
    "Happy": "Creative Writing",
    "Sad": "Dancing",
    "Angry": "Meditation",
    "Fearful": "Breathwork",
    "Disgust": "Aromatherapy",
    "Surprised": "Camping"
}

data = pd.read_csv("activity_recommendations_150.csv")

data["Text_Features"] = data["Description"] + " " + data["Keywords"]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data["Text_Features"])

predicted_emotion = "Fearful"
final_recommendations = get_similar_activities(predicted_emotion)
print(final_recommendations)

[{'Activity_Name': 'Nature Walk', 'Description': 'Leisurely walking to enjoy natural surroundings'}, {'Activity_Name': 'Meditation', 'Description': 'Practice of mindfulness and mental clarity'}, {'Activity_Name': 'Stargazing', 'Description': 'Observing celestial objects at night'}]


In [51]:
index_html = '''
<!doctype html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=0.75">
    <title>Speech Emotion Recognition</title>
    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
    <link rel="icon" href="{{url_for('static', filename='logo.png')}}" type="image/icon type">
    <style>
        body {
            background: url("{{url_for('static', filename='22.png')}}") no-repeat center center fixed;
            background-size: cover;
            font-family: 'Roboto', sans-serif;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            height: 100vh;
            margin: 0;
            color: #fff;
            padding: 0 15px;
            position: relative;
            transition: background 0.4s ease;
        }

        body::before {
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            bottom: 0;
            background: rgba(0, 0, 0, 0.3);
            z-index: -1;
        }

        img {
            max-width: 100%;
            height: 120px;
            margin-bottom: 15px;
        }

        h1 {
            font-size: 2.5em;
            margin-bottom: 12px;
            font-weight: bold;
            background: linear-gradient(#4ea8d2, #4a90e2);
            -webkit-background-clip: text;
            color: transparent;
            text-align: center;
        }

        p.desc {
            font-size: 1em;
            text-align: center;
            margin: 0 0 25px;
            color: #fff;
            line-height: 1.5;
        }

      upload-btn {
          background-color: #4ea8d2;
          border: none;
          border-radius: 8px;
          color: white;
          padding: 10px 25px;
          font-size: 1em;
          cursor: pointer;
          transition: background 0.3s, transform 0.3s;
          width: 80%;
          margin-top: 5px;
          text-align: center; /* Centers the text inside the <p> */
          display: inline-block;
          box-sizing: border-box;
      }

      .upload-container {
          background: rgba(30, 30, 40, 0.9);
          border-radius: 12px;
          box-shadow: 0 6px 30px rgba(0, 0, 0, 0.6);
          padding: 35px;
          text-align: center;
          width: 90%;
          max-width: 400px;
          animation: fadeIn 0.5s ease-in;
          display: flex;
          flex-direction: column;
          align-items: center;
          justify-content: center;
      }

        .predict-btn {
            background-color: #4ea8d2;
            border: none;
            border-radius: 8px;
            color: white;
            padding: 12px 25px;
            font-size: 1.1em;
            cursor: pointer;
            transition: background 0.3s, transform 0.3s;
            width: 100%;
            margin-top: 20px;
        }

        .predict-btn:hover {
            background-color: #3d7ab6;
            transform: scale(1.05);
        }

        .upload-area {
            border: 2px dashed #4ea8d2;
            border-radius: 10px;
            padding: 25px;
            margin: 10px 0;
            transition: background 0.3s;
            color: #4ea8d2;
        }

        .upload-area:hover {
            background-color: rgba(74, 144, 226, 0.1);
        }

        input[type="file"] {
            display: none;
        }

        #fileFeedback {
            display: none;
            color: #4ea8d2;
            margin-top: 12px;
        }

        #toast {
            visibility: hidden;
            min-width: 250px;
            margin: 10px auto;
            background-color: #ff0000;
            color: #fff;
            text-align: center;
            border-radius: 2px;
            padding: 16px;
            position: fixed;
            z-index: 1;
            left: 50%;
            transform: translateX(-50%);
            bottom: 30px;
            font-size: 15px;
            transition: visibility 0s, opacity 0.5s linear;
        }

        #toast.show {
            visibility: visible;
            opacity: 1;
        }

        .loader-container {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background: rgba(0, 0, 0, 0.7);
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            z-index: 1000;
            opacity: 0;
            visibility: hidden;
            transition: opacity 0.3s, visibility 0.3s;
        }

        .loader-container.active {
            opacity: 1;
            visibility: visible;
        }

        .loader {
            width: 100px;
            height: 100px;
            margin-bottom: 2rem;
            position: relative;
        }

        .loader:before {
            content: '';
            position: absolute;
            width: 100%;
            height: 100%;
            border-radius: 50%;
            border: 4px solid transparent;
            border-top-color: #4a90e2;
            border-right-color: #3d7ab6;
            border-bottom-color: #2a5a8e;
            animation: spin 1.5s linear infinite, gradientSpin 3s ease-in-out infinite;
        }

        .loader-text {
            color: #fff;
            font-family: 'Roboto', sans-serif;
            font-size: 1.2rem;
            text-align: center;
            opacity: 0;
            transform: translateY(20px);
            transition: opacity 0.5s, transform 0.5s;
        }

        .loader-text.active {
            opacity: 1;
            transform: translateY(0);
        }

        @keyframes spin {
            to {
                transform: rotate(360deg);
            }
        }

        @keyframes gradientSpin {
            0% {
                border-top-color: #4a90e2;
                border-right-color: #3d7ab6;
                border-bottom-color: #2a5a8e;
            }
            33% {
                border-top-color: #2a5a8e;
                border-right-color: #4a90e2;
                border-bottom-color: #3d7ab6;
            }
            66% {
                border-top-color: #3d7ab6;
                border-right-color: #2a5a8e;
                border-bottom-color: #4a90e2;
            }
            100% {
                border-top-color: #4a90e2;
                border-right-color: #3d7ab6;
                border-bottom-color: #2a5a8e;
            }
        }

        @keyframes fadeIn {
            from {
                opacity: 0;
            }
            to {
                opacity: 1;
            }
        }

        @media (max-width: 576px) {
            h1 {
                font-size: 1.8em;
            }

            p.desc {
                font-size: 0.9em;
            }

            .upload-container {
                padding: 15px;
            }

            .upload-btn {
                padding: 8px 18px;
                font-size: 1em;
            }

            .predict-btn {
                padding: 10px 20px;
                font-size: 1.1em;
            }
        }
    </style>
</head>
<body>
    <img src="{{url_for('static', filename='logo.png')}}" alt="Logo"/>
    <h1>Speech Emotion Recognition & Activity Recommendation</h1>
    <p class="desc">Upload an audio file and our AI will analyze the emotional content of speech and recommend activities based on your mood!</p>
    <div class="upload-container">
        <form method="post" enctype="multipart/form-data" onsubmit="return handleSubmit(event);">
            <div class="upload-area">
                <label for="file" class="upload-label">
                    <p>Drag and drop your audio file here or</p>
                    <p class="upload-btn">Click to upload file</p>
                </label>
                <input type="file" name="file" accept="audio/*" id="file" required onchange="updateUploadStatus(event);">
                <div id="fileFeedback"></div>
            </div>
            <button class="predict-btn" type="submit" id="predictEmotionBtn">Predict Emotion</button>
        </form>
        <div id="toast"></div>
    </div>

    <div class="loader-container">
        <div class="loader"></div>
        <div class="loader-text">Uploading File...</div>
    </div>

<script>
    function updateUploadStatus(event) {
        const feedback = document.getElementById('fileFeedback');

        if (event.target.files.length > 0) {
            feedback.textContent = "File selected: " + event.target.files[0].name;
            feedback.style.display = "block";
        } else {
            feedback.style.display = "none";
        }
    }

    function showLoader() {
        const container = document.querySelector('.loader-container');
        const text = document.querySelector('.loader-text');

        container.classList.add('active');
        text.classList.add('active');

        setTimeout(() => {
            text.style.opacity = '0';
            setTimeout(() => {
                text.textContent = 'Predicting Emotion...';
                text.style.opacity = '1';
            }, 500);
        }, 2000);
    }

    function showToast(message) {
        const toast = document.getElementById('toast');
        toast.textContent = message;
        toast.className = "show";
        toast.style.visibility = 'visible';
        setTimeout(() => {
            toast.className = toast.className.replace("show", "");
            toast.style.visibility = 'hidden';
        }, 3000);
    }

    function handleSubmit(event) {
        const fileInput = document.getElementById('file');
        if (fileInput.files.length === 0) {
            event.preventDefault();
            showToast('Please upload a file first!');
            return false;
        }
        showLoader();
        return true;
    }

    const uploadArea = document.querySelector('.upload-area');

    uploadArea.addEventListener('dragover', (e) => {
        e.preventDefault();
        uploadArea.style.backgroundColor = 'rgba(74, 144, 226, 0.2)';
    });

    uploadArea.addEventListener('dragleave', () => {
        uploadArea.style.backgroundColor = '';
    });

    uploadArea.addEventListener('drop', (e) => {
        e.preventDefault();
        const files = e.dataTransfer.files;
        const fileInput = document.getElementById('file');
        fileInput.files = files;
        updateUploadStatus({ target: fileInput });
        uploadArea.style.backgroundColor = '';
    });
</script>
</body>
</html>
'''

In [20]:
result_html = '''
<!doctype html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Emotion Recognized</title>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap" rel="stylesheet">
    <link rel="icon" href="{{url_for('static', filename='logo.png')}}" type="image/icon type">
    <style>
        body {
            background: url("{{url_for('static', filename='22.png')}}") no-repeat center center fixed;
            background-size: cover;
            font-family: 'Montserrat', sans-serif;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            height: 100vh;
            margin: 0;
            padding: 0;
            color: #f0f0f0;
            box-sizing: border-box;
        }

        header {
            text-align: center;
            margin-bottom: 30px; /* Slightly reduced margin */
        }

        h1 {
            font-size: 3em; /* Slightly smaller font size */
            font-weight: 700;
            margin: 0;
            color: #4ea8d2;
        }

        p {
            font-size: 1.2em; /* Slightly smaller font size */
            margin-top: 10px;
            color: #d3d3d3;
        }

        img {
            height: 90px; /* Slightly smaller logo size */
            margin-bottom: 20px;
        }

        .container {
            display: flex;
            justify-content: space-between;
            align-items: flex-start;
            gap: 40px; /* Slightly reduced gap */
            width: 100%;
            max-width: 1400px;
        }

        .result-container,
        .activities-container {
            background: rgba(30, 40, 60, 0.8);
            border-radius: 15px;
            box-shadow: 0px 8px 20px rgba(0, 0, 0, 0.6); /* Slightly reduced shadow */
            padding: 30px; /* Slightly reduced padding */
            text-align: center;
            width: 48%;
            color: #f0f0f0;
        }

        .result-container:hover,
        .activities-container:hover {
            transform: translateY(-5px);
            box-shadow: 0px 12px 25px rgba(0, 0, 0, 0.7);
        }

        audio {
            margin-top: 15px;
            width: 100%;
            border-radius: 8px;
            background: transparent;
        }

        .try-again-btn {
            background: #4ea8d2;
            border: none;
            padding: 12px 28px; /* Slightly reduced padding */
            border-radius: 30px;
            color: white;
            font-size: 1em; /* Slightly reduced font size */
            cursor: pointer;
            margin-top: 25px; /* Slightly reduced margin */
            font-weight: 600;
            transition: background 0.3s, transform 0.3s;
        }

        .try-again-btn:hover {
            background: #2e8aaf;
            transform: translateY(-3px);
            box-shadow: 0px 8px 20px rgba(0, 0, 0, 0.6);
        }

        h2 {
            font-size: 1.7em; /* Slightly reduced font size */
            margin-bottom: 18px; /* Slightly reduced margin */
            font-weight: 600;
            color: #ffffff;
        }

        table {
            width: 100%;
            border-collapse: collapse;
            margin-top: 10px;
        }

        th,
        td {
            padding: 14px; /* Slightly reduced padding */
            text-align: left;
            border: 1px solid rgba(77, 168, 218, 0.6);
            font-size: 1em; /* Slightly reduced font size */
        }

        th {
            background-color: #4ea8d2;
            color: white;
            text-align: center;
            text-transform: uppercase;
            letter-spacing: 1px;
        }

        tr:nth-child(even) {
            background-color: rgba(30, 45, 70, 0.8);
        }

        tr:nth-child(odd) {
            background-color: rgba(20, 35, 55, 0.8);
        }

        tr:hover {
            background-color: rgba(77, 168, 218, 0.2);
        }

        footer {
            margin-top: 40px; /* Slightly reduced margin */
            text-align: center;
            font-size: 0.9em; /* Slightly reduced font size */
            color: #e0e0e0;
        }

        footer a {
            color: #4ea8d2;
            text-decoration: none;
        }

        footer a:hover {
            text-decoration: underline;
        }
    </style>
</head>

<body>
    <header>
        <img src="{{ url_for('static', filename='logo.png') }}" alt="Logo" />
        <h1>Emotion Recognized</h1>
        <p>Emotion: <strong>{{ emotion }}</strong> | Probability: <strong>{{ probs }}%</strong></p>
    </header>

    <div class="container">
        <div class="result-container">
            <h2>Audio Result</h2>
            <audio controls>
                <source src="{{ url_for('uploaded_file', filename=audio_filename) }}" type="audio/wav">
                Your browser does not support the audio tag.
            </audio>
            <form method="get" action="/">
                <button class="try-again-btn" type="submit">Try Again</button>
            </form>
        </div>

        <div class="activities-container">
            <h2>Recommended Activities</h2>
            <table>
                <thead>
                    <tr>
                        <th>Activity Name</th>
                        <th>Description</th>
                    </tr>
                </thead>
                <tbody>
                    {% for activity in activities1 %}
                    <tr>
                        <td>{{ activity['Activity_Name'] }}</td>
                        <td>{{ activity['Description'] }}</td>
                    </tr>
                    {% endfor %}
                </tbody>
            </table>
        </div>
    </div>

    <footer>
        <p>Powered by <a href="https://www.instagram.com/load_thecode/">load_thecode</a> | © 2024 All rights reserved</p>
    </footer>
</body>

</html>
'''

In [52]:
from flask import Flask, render_template, request, redirect, send_from_directory, render_template_string
from werkzeug.utils import secure_filename
import os


app = Flask(__name__)

app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['ALLOWED_EXTENSIONS'] = {'wav', 'mp3'}

if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

live_prediction = LivePredictions()

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            # This handles the case when no file part is present
            return redirect(request.url)

        file = request.files['file']

        if file.filename == '':
            # This handles the case when no file is selected
            return redirect(request.url)

        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)

            predicted_emotion, prob = live_prediction.make_predictions(file=filepath)
            activites = get_similar_activities(predicted_emotion)
            return render_template_string(result_html, emotion=predicted_emotion, probs=prob, audio_filename=filename, activities1=activites)
            # return render_template("result_2.html", emotion=predicted_emotion, probs=prob, audio_filename=filename, activities1=activites)

    return render_template_string(index_html)
    # return render_template("index_copy.html")

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

# Run the Flask app
port = 5000
app.run(host='0.0.0.0', port=port)
public_url = ngrok.connect(port)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")
app.run(host='0.0.0.0', port=port)



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


 * ngrok tunnel "NgrokTunnel: "https://b798-34-125-245-48.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Dec/2024 12:15:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2024 12:15:51] "GET /static/logo.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2024 12:15:52] "GET /static/22.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2024 12:15:55] "GET /static/logo.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2024 12:17:23] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2024 12:17:23] "GET /static/logo.png HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2024 12:17:35] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2024 12:17:36] "GET /static/logo.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2024 12:17:36] "GET